In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [2]:
# Load a different time series dataset (you can replace this with your own dataset) - Example dataset: Daily minimum temperature in Melbourne
data = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv")

In [3]:
# Specify the date column and target column
date_column = "Date"
target_column = "Temp"

In [4]:
# Convert the "Date" column to a datetime format
data[date_column] = pd.to_datetime(data[date_column])
data.shape

(3650, 2)

In [7]:
np.random.seed(0)
# Normalize the data using Min-Max scaling
scaler = MinMaxScaler()
data[target_column] = scaler.fit_transform(data[[target_column]])

# Prepare the data for LSTM
sequence_length = 12  # Number of time steps to look back
X, y = [], []

for i in range(len(data) - sequence_length):
    X.append(data[target_column].iloc[i:i+sequence_length].values)
    y.append(data[target_column].iloc[i+sequence_length])

X, y = np.array(X), np.array(y)

# Split the data into training and testing sets
train_size = int(len(data) * 0.8)
X_train, X_test, y_train, y_test = X[:train_size], X[train_size:], y[:train_size], y[train_size:]

In [8]:
# Build LSTM model
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation="relu", input_shape=(sequence_length, 1)))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss="mean_absolute_error")

In [9]:
# Train the LSTM model
model_lstm.fit(X_train, y_train, epochs=100, batch_size=16, verbose=2)

Epoch 1/100
183/183 - 3s - loss: 0.1538 - 3s/epoch - 16ms/step
Epoch 2/100
183/183 - 2s - loss: 0.0838 - 2s/epoch - 9ms/step
Epoch 3/100
183/183 - 2s - loss: 0.0828 - 2s/epoch - 9ms/step
Epoch 4/100
183/183 - 2s - loss: 0.0823 - 2s/epoch - 8ms/step
Epoch 5/100
183/183 - 1s - loss: 0.0811 - 1s/epoch - 6ms/step
Epoch 6/100
183/183 - 1s - loss: 0.0802 - 1s/epoch - 6ms/step
Epoch 7/100
183/183 - 1s - loss: 0.0800 - 1s/epoch - 6ms/step
Epoch 8/100
183/183 - 1s - loss: 0.0797 - 1s/epoch - 6ms/step
Epoch 9/100
183/183 - 1s - loss: 0.0776 - 1s/epoch - 6ms/step
Epoch 10/100
183/183 - 1s - loss: 0.0779 - 1s/epoch - 6ms/step
Epoch 11/100
183/183 - 1s - loss: 0.0763 - 1s/epoch - 6ms/step
Epoch 12/100
183/183 - 1s - loss: 0.0762 - 1s/epoch - 7ms/step
Epoch 13/100
183/183 - 2s - loss: 0.0753 - 2s/epoch - 13ms/step
Epoch 14/100
183/183 - 3s - loss: 0.0756 - 3s/epoch - 14ms/step
Epoch 15/100
183/183 - 1s - loss: 0.0754 - 1s/epoch - 6ms/step
Epoch 16/100
183/183 - 1s - loss: 0.0749 - 1s/epoch - 6ms/ste

In [10]:
# Predict using LSTM model
y_pred_lstm = model_lstm.predict(X_test)

23/23 [==============================] - 0s 3ms/step


In [11]:
# Inverse transform the predictions to the original scale
y_pred_lstm = scaler.inverse_transform(y_pred_lstm)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))